In [ ]:
%load_ext autoreload
%autoreload 2

# Global Summary Creator

In [ ]:
from Code.Utils.util_methods import NNUtils
import os
import json
from tqdm import tqdm
import pandas as pd

base = NNUtils.find_project_root(os.getcwd())
base

In [ ]:
MAIN_DIRECTORY = f'{base}/Code/Full_systems/Selfies_Mol/Multitask_Classifier/Models'
OUTPUT_FILE = f'{MAIN_DIRECTORY}/global_summary.csv'

## DON'T TOUCH FROM HERE

In [ ]:
def iterate_through_folders(directory):
    """
    Iterates through all folders in the given directory and extracts the information from the evaluation.json and parameters.json files
    :param directory: the directory to iterate through
    :return: a list of dictionaries containing the information from the .json files
    """
    # get all directories and their creation times
    folders = [(root, os.path.getctime(root)) for root, dirs, files in os.walk(directory)]
    # sort folders by creation time from oldest to newest
    folders.sort(key=lambda x: x[1])

    results = []
    for folder, _ in tqdm(folders):
        files = os.listdir(folder)
        instance = {}
        if '.done' in files:
            for file in files:
                if file.endswith('.json') and 'evaluation' in file:
                    file_path = os.path.join(folder, file)
                    with open(file_path, 'r') as f:
                        data = json.load(f)
                    instance['model_name'] = file_path.split('/')[-2]
                    instance['architecture'] = '_'.join(instance['model_name'].split('_')[0:len(instance['model_name'].split('_'))-2])
                    instance['>=0.6'] = data['prediction_rates']['60%+']
                    instance['>=0.9'] = data['prediction_rates']['90%+']
                    instance['>=1'] = data['prediction_rates']['100%+']
                    instance['correct_prediction'] = data['prediction_rates']['correct']
                elif file.endswith('.json') and 'parameters' in file:
                    file_path = os.path.join(folder, file)
                    with open(file_path, 'r') as f:
                        data = json.load(f)
                    instance['layer_sizes'] = data['layer_sizes']
                    instance['dropouts'] = data['dropouts']
                    instance['learning_rate'] = data['learning_rate']
                    instance['batch_size'] = data['batch_size']
        if instance != {}:
            results.append(instance)
    return results

In [ ]:
summaries = iterate_through_folders(MAIN_DIRECTORY)
#summaries

In [ ]:
summaries_df = pd.DataFrame(summaries)
summaries_df

In [ ]:
summaries_df.to_csv(OUTPUT_FILE)